### Bi-Metrics
#### Por Andrea Fuentes Pérez y Víctor Sánchez Auñón
#### 3º de Ingeniería Biomédica

VALIDACIÓN Y FORMATO DEL MRZ CON PRE-PROCESIONAMIENTO:

In [143]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from PIL import Image, ImageEnhance
import os

mrz = {
    "linea1": "",
    "linea2": "",
    "linea3": ""
}

model = ocr_predictor(pretrained=True)

def extract_text(ocr_model, image_file_path: str, angulo: float = 0, 
                 algoritmo: Image.Resampling = Image.Resampling.NEAREST, idCorte: int = 0):
    """Extrae el texto de una imagen a partir de un modelo OCR"""
    
    # Abrimos, cropeamos, giramos y aumentamos para mejor lectura
    img = Image.open(image_file_path).convert('L')

    ancho, alto = img.size
    # (inicioX, inicioY, finX, finY)
    corte = [(0, 0, ancho, alto), #0 imagen completa
             (0, alto*0.6, ancho, alto), #1 parte inferior (MRZ)
             (0, alto*0.18, ancho*0.3, alto*0.28), #2 DNI
             (ancho*0.35, alto*0.25, ancho*0.6, alto*0.64), #3 Apellidos
             (ancho*0.35, alto*0.37, ancho*0.6, alto*0.6), #4 Nombre
             (0, alto*0.6, ancho, alto), #1 parte inferior (MRZ)
             (0, alto*0.6, ancho, alto),] #1 parte inferior (MRZ)
    img = img.crop(corte[idCorte])
    img = img.rotate(angulo, resample=algoritmo, expand=True)  
    img = ImageEnhance.Contrast(img).enhance(2.0)    
    temp_path = "temp_mrz_process.jpg"
    img.save(temp_path)

    doc = DocumentFile.from_images(temp_path)
    result = ocr_model(doc)
    
    # Limpiar archivo temporal
    if os.path.exists(temp_path):
        os.remove(temp_path)
        
    return result

def show_ocr_result(result):
    """Muestra una imagen de resultado y el texto extraido"""

    # mostrar resultado
    result.show()

    # mostrar texto con mas de 50% de confianza
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                words = []
                for word in line.words:
                    if word.confidence > 0.5:
                        words.append(word.value)
                
                
                print(' '.join(words).upper())

#Primera estraregia: Algoritmo BICUBIC, angulo de 0.5 para la línea 1 y 2, 0.3 para línea 3
def leerMRZ(path):
    text_back1 = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.BICUBIC, idCorte=1)
    text_back2 = extract_text(model, path, angulo=0.3, algoritmo=Image.Resampling.BICUBIC, idCorte=1)

    mrz = {
        "linea1": text_back1.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back1.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back2.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta. Probando con el siguiente algoritmo...\n")
        estrategia2MRZ(path)
    else:
        return mrz

#Segunda estrategia: Algoritmo NEAREST, angulo de 0.5 para las tres líneas
def estrategia2MRZ(path):
    text_back = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.NEAREST, idCorte=1)

    mrz = {
        "linea1": text_back.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta. Probando con el siguiente algoritmo...\n")
        estrategia3MRZ(path)
    else:
        return mrz

#Tercera estrategia: Algoritmo BILINEAL, angulo de 0.5 para las tres líneas
def estrategia3MRZ(path):
    text_back = extract_text(model, path, angulo=0.5, algoritmo=Image.Resampling.BILINEAR, idCorte=1)

    mrz = {
        "linea1": text_back.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta, por favor repita la imagen\n")
    else:
        return mrz

In [103]:
#Obtenemos MRZ de una imagen
mrz = leerMRZ('./dni_fotos/vback.jpg')

linea1: IDESPCIU151694451256526Z<<<<<<
linea2: 0511058M2909107ESP<<<<<<<<<<<7
linea3: SANCHEZ<AUNON<<VICTOR<<<<<<<<<


CÁLCULO Y COMPROBACIÓN DE CHECKSUMS:

In [104]:
for key, value in mrz.items():
    print(f"{key}: {value}")
print("\n")

num_docu = mrz["linea1"][5:14]
fecha_nac = mrz["linea2"][0:6]
#control es L1 sin ID ni pais (5 al final) + L2 sin sexo ni nacionalidad (0 a 7 + 8 a 15 + 18 a 29)
control = mrz["linea1"][5:] + mrz["linea2"][0:7] + mrz["linea2"][8:15] + mrz["linea2"][18:29]
def calculoHash(campo):
    total = 0
    posi = 0 #empieza en 0 -> *7, posi++ 1 -> *3, posi++ 2 -> *1 y posi - 2 
    valoresHash = [7, 3, 1]
    for char in campo:
        total += valorCharHash(char) * valoresHash[posi]
        
        if posi == 2:
            posi = 0
        else:
            posi += 1
    return total % 10 # cheksum = resto de 10 (es el ultimo digito)

def valorCharHash(char):
    if char.isdigit():
        return int(char)
    elif char == "<":
        return 0
    else:
        # A=65, si tiene q valer 10, entonces ord(A) - 55 = 10
        return ord(char) - 55

print("Hash numero documento: ", calculoHash(num_docu)) #4
print("Hash fecha nacimiento: ", calculoHash(fecha_nac)) #8
print("Hash control final: ", calculoHash(control)) #7


linea1: IDESPCIU151694451256526Z<<<<<<
linea2: 0511058M2909107ESP<<<<<<<<<<<7
linea3: SANCHEZ<AUNON<<VICTOR<<<<<<<<<


Hash numero documento:  4
Hash fecha nacimiento:  8
Hash control final:  7


VERIFICACIÓN DE LA CONSISTENCIA INTERNA

In [147]:
def leerApellidos(path):
    text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=3)

    apellidos = {
            "apellido1": text_front.pages[0].blocks[0].lines[1].words[0].value,
            "apellido2": text_front.pages[0].blocks[0].lines[2].words[0].value,
        }

    return apellidos
def leerNombre(path):
    text_front = extract_text(model, path, angulo=2, algoritmo=Image.Resampling.BICUBIC, idCorte=4)
    nombre = {
            "nombre": text_front.pages[0].blocks[0].lines[1].words[0].value,
        }
    return nombre
def leerNombreCompleto(path):
    nombre = leerNombre(path)
    apellidos = leerApellidos(path)

    nombre_completo = {
        "nombre": nombre["nombre"],
        "apellido1": apellidos["apellido1"],
        "apellido2": apellidos["apellido2"]
    }
    return nombre_completo

In [150]:
path = "./dni_fotos/vfront.jpg"
nombre_completo = leerNombreCompleto(path)
for key, val in nombre_completo.items():
    print(key,val)
print("\n")
path = "./dni_fotos/afront.jpg"
nombre_completo = leerNombreCompleto(path)
for key, val in nombre_completo.items():
    print(key,val)
print("\n")
path = "./dni_fotos/front.jpg"
nombre_completo = leerNombreCompleto(path)
for key, val in nombre_completo.items():
    print(key,val)

nombre VICTOR
apellido1 SANCHEZ
apellido2 AUNON


nombre ANDREA
apellido1 FUENTES
apellido2 PEREZ


nombre CARMEN
apellido1 ESPANOLA
apellido2 ESPANOLA
